In [4]:
import tensorflow as tf
import numpy as np
import cv2
import dataset
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
train_path = '/home/kaustabh/Python_Projects/traindata'
test_path = '/home/kaustabh/test'

img_size = 64

classes = ['cats', 'dogs']
num_classes = len(classes)

In [6]:
data = dataset.read_train_sets(train_path, img_size, classes)

Reading training images
Loading cats files (Index: 0)
Loading dogs files (Index: 1)


In [7]:
def initialize_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev = 0.05))

def initialize_biases(length):
    return tf.Variable(tf.constant(0.05, shape = length))

In [8]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    return tf.nn.relu( tf.nn.bias_add(x, b) )

def maxpool(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')

In [9]:
weights = {
    'wc1': initialize_weights([5, 5, 3, 32]),
    'wc2': initialize_weights([5, 5, 32, 64]),
    'wc3': initialize_weights([5, 5, 64, 32]),
    
    'wd1': initialize_weights([8 * 8 * 32, 1024]),
    'out': initialize_weights([1024, num_classes])
}

biases = {
    'bc1': initialize_biases([32]),
    'bc2': initialize_biases([64]),
    'bc3': initialize_biases([32]),
    
    'bd1': initialize_biases([1024]),
    'out': initialize_biases([num_classes]),
}

In [10]:
def convnet(x, weights, biases, dropout):
    
    x = tf.reshape(x, shape = [-1, 64, 64, 3])
    
    #convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool(conv1)
    
    #convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool(conv2)
    
    #convolution layer 3
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool(conv3)
    
    #fully connected layer 1
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [11]:
#defining placeholders

X = tf.placeholder(tf.float32, shape = [None, 64 * 64 * 3], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_classes], name = 'Y')
keep_prob = tf.placeholder(tf.float32, name = 'dropout')

In [12]:
learning_rate = 1e-4
epochs = 10000
batch_size = 100
display_step = 100

In [13]:
#constuct model
Y_pred = convnet(X, weights, biases, keep_prob)

#define cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Y_pred, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

#evaluate model
correct_pred = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#initialize global variable
init = tf.global_variables_initializer()

In [22]:
def print_progress(epoch, feed_dict_train, session):
    # Calculate the accuracy on the training-set.
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    msg = "Epoch {0} --- Training Accuracy: {1:>6.1%}"
    print(msg.format(epoch + 1, acc))

In [23]:
total_iterations = 0
def optimize(num_iterations):
    
    global total_iterations
    
    with tf.Session() as sess:
        sess.run(init)
        for i in range(total_iterations, total_iterations + num_iterations):
        
            x_batch, y_batch, _, cls_batch = data.train.next_batch(batch_size)
            x_batch = x_batch.reshape(batch_size, 64 * 64 * 3)
            feed_dict_train = {X: x_batch, Y: y_batch, keep_prob: 0.75}
            feed_dict_train_eval = {X: x_batch, Y: y_batch, keep_prob: 1}
            sess.run(optimizer, feed_dict = feed_dict_train)
            
            if i % int(data.train.num_examples/batch_size) == 0: 
                epoch = int(i / int(data.train.num_examples/batch_size))
                print_progress(epoch, feed_dict_train_eval, sess)
                
            
            total_iterations += num_iterations         

In [24]:
optimize(num_iterations = 10000)

Epoch 1 --- Training Accuracy:  51.0%
Epoch 2 --- Training Accuracy:  74.0%
Epoch 3 --- Training Accuracy:  76.0%
Epoch 4 --- Training Accuracy:  77.0%
Epoch 5 --- Training Accuracy:  74.0%
Epoch 6 --- Training Accuracy:  77.0%
Epoch 7 --- Training Accuracy:  77.0%
Epoch 8 --- Training Accuracy:  80.0%
Epoch 9 --- Training Accuracy:  80.0%
Epoch 10 --- Training Accuracy:  82.0%
Epoch 11 --- Training Accuracy:  83.0%
Epoch 12 --- Training Accuracy:  87.0%
Epoch 13 --- Training Accuracy:  87.0%
Epoch 14 --- Training Accuracy:  89.0%
Epoch 15 --- Training Accuracy:  90.0%
Epoch 16 --- Training Accuracy:  90.0%
Epoch 17 --- Training Accuracy:  91.0%
Epoch 18 --- Training Accuracy:  93.0%
Epoch 19 --- Training Accuracy:  94.0%
Epoch 20 --- Training Accuracy:  93.0%
Epoch 21 --- Training Accuracy:  95.0%


KeyboardInterrupt: 